<h1 style="text-align:center">Part 1: Web Scrapping and Data Cleaning</h1>

In [10]:
import selenium


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
driver.get(url)
driver.page_source


Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
File found in cache by path [C:\Users\Kevin\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


'<html class="client-js" lang="en" dir="ltr"><head>\n<meta charset="UTF-8">\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"d378e050-0b55-4b38-a8c4-d4c0f7bb8a49","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgPa

In [97]:
wikiPage = driver.page_source

from bs4 import BeautifulSoup

soup = BeautifulSoup(wikiPage)

#print(soup.find("tbody"))
#print("--------------------")
#print(soup.prettify())

table = soup.findChildren("tbody")[0]
rows = table.findChildren(["tr"])

dictToronto = {"Postal Code":[], "Borough":[], "Neighborhood":[]}
columnNames = ["Postal Code", "Borough", "Neighborhood"]
for i in range(len(rows)):
    
    cells = rows[i].findChildren(["td"])
    
    for j in range(len(cells)):
        dictToronto[columnNames[j]].append(cells[j].next_element)
    
import pandas as pd

dfToronto = pd.DataFrame.from_dict(dictToronto)

In [117]:
"""Data Cleaning"""

# First we erase trailing spaces
dfToronto = dfToronto.apply(lambda x: x.str.strip())

In [162]:
import numpy as np

# We drop rows that have no Borough assigned
dfToronto = dfToronto.replace("Not assigned", np.nan).dropna(axis = 0, subset=["Borough"])

# Reset index
dfToronto.reset_index(drop=True, inplace = True)

# We check if there is any Neighborhood with value not assigned
print("Neighborhoods with no assigned value: ", (dfToronto["Neighborhood"].isnull()).sum())

Neighborhoods with no assigned value:  0


In [163]:
# We print the shape
print("Shape of the final DataFrame: ", dfToronto.shape)
dfToronto

Shape of the final DataFrame:  (103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
